In [1]:
# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

- [06 PyTorch에서 다른 모델의 매개변수를 사용하여 빠르게 모델 시작하기(WARM START)](https://tutorials.pytorch.kr/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)
- 모델을 부분적으로 불러오는 것은 학습 전이(Transfer learning)나 복잡한 모델을 새로 학습할 때 자주 접하는 시나리오임
- 학습된 매개변수를 활용하면 학습 과정을 빠르게 시작(warmstart)할 수 있음
- 그러면 모델을 처음부터 훈련시킬 때보다 훨씬 일찍 수렴하리라 기대할 수 있음
- 이는 활용할 수 있는 매개변수가 얼마 안 될 때에도 마찬가지임

# 도입
- 일부 키가 누락된 부분적인 `state_dict`를 불러올 때든 아니면 결과를 저장할 모델보다 키가 많은 `state_dict`를 불러올 때든 `load_state_dict()` 함수의 인자인 `strict`를 `False`로 두면 매치되지 않는 키를 무시하게 할 수 있음
- 다른 모델의 매개변수를 사용하여 모델을 빠르게 시작하는 실험을 진행해 볼 것임

# 단계
1. 데이터를 불러오는데 필요한 모든 라이브러리를 import 하기
2. 신경망 A와 B를 정의하고 초기화하기
3. 모델 A를 저장하기
4. 모델 B로 모델을 불러오기

## 데이터를 불러오는데 필요한 모든 라이브러리를 import 하기
- `torch`와 여기 포함된 `torch.nn`과 `torch.optim`을 사용할 것임

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

## 신경망 A와 B를 정의하고 초기화하기
- 하나의 예로써 이미지를 학습하는 신경망을 만들어 볼 것임 
- 여기서는 신경망을 두 개 만들려고 하며, 신경망 A의 매개변수를 신경망 B로 불러올 것임

In [ ]:
class NetA(nn.Module):
    def __init__(self):
        super(NetA, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # in_channel, out_channel, kernel_size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)